### Konstantina Andronikou

## Transitive Vs Intransitive Verbs 

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

## Intransitive 

In [4]:
# remove warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("Carla pays.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Carla pays."]
pred = predict_and_conf(d)

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [25]:
editor = Editor()

# create examples
t = editor.template('{first_name} pays.', meta=True, nsamples=100, remove_duplicates = True)

# for k, v in t.items():
#     print(k,v)

In [26]:
with open('Dataset_BERT/Intransitive.txt', 'w') as f:
    print(t.data, file = f)

In [27]:
test = MFT(**t, name = 'detect_arg0name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Intransitive_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
#         print(case,exp)

Predicting 100 examples
Test cases:      100
Fails (rate):    76 (76.0%)

Example fails:
[ARG1: Bill] [V: pays] .
----
[ARG1: Diana] [V: pays] .
----
[ARG1: Alice] [V: pays] .
----


In [28]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_Intransitive_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive 

In [29]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [30]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [31]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [32]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [33]:
editor = Editor()

# create examples
t = editor.template('{first_name} pays for coffee.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

# for k, v in t.items():
#     print(k, v)

In [34]:
with open('Dataset_BERT/Transitive.txt', 'w') as f:
    print(t.data, file = f)

In [35]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Transitive_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        #print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Cynthia', 'pays', 'for', 'coffee', '.'] [ True]
['Eleanor', 'pays', 'for', 'coffee', '.'] [ True]
['Harry', 'pays', 'for', 'coffee', '.'] [ True]
['Mary', 'pays', 'for', 'coffee', '.'] [ True]
['Keith', 'pays', 'for', 'coffee', '.'] [ True]
['Matthew', 'pays', 'for', 'coffee', '.'] [ True]
['Lawrence', 'pays', 'for', 'coffee', '.'] [ True]
['Bobby', 'pays', 'for', 'coffee', '.'] [ True]
['Edward', 'pays', 'for', 'coffee', '.'] [ True]
['Jill', 'pays', 'for', 'coffee', '.'] [ True]
['Steven', 'pays', 'for', 'coffee', '.'] [ True]
['Roger', 'pays', 'for', 'coffee', '.'] [ True]
['Philip', 'pays', 'for', 'coffee', '.'] [ True]
['Carol', 'pays', 'for', 'coffee', '.'] [ True]
['Jack', 'pays', 'for', 'coffee', '.'] [ True]
['Ron', 'pays', 'for', 'coffee', '.'] [ True]
['George', 'pays', 'for', 'coffee', '.'] [ True]
['Pamela', 'pays', 'for', 'coffee', '.'] [ True]
['Richard', 'pays', 'for', 'coffee', '.'] [ True]
['Ke

['Ray', 'pays', 'for', 'coffee', '.'] [ True]
['Claire', 'pays', 'for', 'coffee', '.'] [ True]
['Samuel', 'pays', 'for', 'coffee', '.'] [ True]
['Gary', 'pays', 'for', 'coffee', '.'] [ True]
['Ian', 'pays', 'for', 'coffee', '.'] [ True]
['Roy', 'pays', 'for', 'coffee', '.'] [ True]
['Lucy', 'pays', 'for', 'coffee', '.'] [ True]
['Leslie', 'pays', 'for', 'coffee', '.'] [ True]
['Caroline', 'pays', 'for', 'coffee', '.'] [ True]
['Rose', 'pays', 'for', 'coffee', '.'] [ True]
['Alice', 'pays', 'for', 'coffee', '.'] [ True]
['Joan', 'pays', 'for', 'coffee', '.'] [ True]
['Diana', 'pays', 'for', 'coffee', '.'] [ True]
['Keith', 'pays', 'for', 'coffee', '.'] [ True]
['Pamela', 'pays', 'for', 'coffee', '.'] [ True]
['Greg', 'pays', 'for', 'coffee', '.'] [ True]
['Dorothy', 'pays', 'for', 'coffee', '.'] [ True]
['Paul', 'pays', 'for', 'coffee', '.'] [ True]
['Kathleen', 'pays', 'for', 'coffee', '.'] [ True]
['Margaret', 'pays', 'for', 'coffee', '.'] [ True]
['Anthony', 'pays', 'for', 'coffee', 

In [36]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_Transitive_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive-Intransitive Robustness Test

In [ ]:
editor = Editor()

# create examples
t = editor.template('{first_name} payed.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

# for k, v in t.items():
#     print(k, v)

In [ ]:
with open('Dataset_BERT/Intransitive_Rob_BERT.txt', 'w') as f:
    print(t.data, file = f)

In [ ]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Intransitive_Rob_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
#         print(case,exp)

In [ ]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_Intransitive_Rob_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive

In [ ]:
editor = Editor()

# create examples
t = editor.template('{first_name} payed for coffee.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

# for k, v in t.items():
#     print(k, v)

In [ ]:
with open('Dataset_BERT/Transitive_Rob_BERT.txt', 'w') as f:
    print(t.data, file = f)

In [ ]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Transitive_Rob_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
#         print(case,exp)

In [ ]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_Transitive_Rob_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 